<a href="https://colab.research.google.com/github/VijayTaori/Lifo_step1/blob/main/Detailed_lifo_btc_step_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import calendar
df_open = pd.read_csv('/content/et_open_alldata.csv')
df_close = pd.read_csv('/content/et_close_alldata.csv')
df_close['Close Datetime'] = pd.to_datetime(df_close['Close_unix_timestamp'],dayfirst=True,unit='s')
df_open['Open Datetime'] = pd.to_datetime(df_open['Open_unix_timestamp'],dayfirst=True,unit='s')
df_close_btc = df_close[df_close['Close Action'] =='Buy Bitcoin']
df_open_btc = df_open[df_open['Open Action'] =='Buy Bitcoin']
df_close_btc_original = df_close_btc
df_open_btc_original = df_open_btc
df_close_btc.reset_index(inplace=True)
df_close_btc = df_close_btc.drop('index', axis=1)
df_close_btc.reset_index(inplace=True)
df_close_btc.rename(columns={'index':'Original_Close_index'},inplace=True)
df_open_btc.reset_index(inplace=True)
df_open_btc = df_open_btc.drop('index', axis=1)
df_open_btc.reset_index(inplace=True)
df_open_btc.rename(columns={'index':'Original_Open_index'},inplace=True)
data_btc = {}
df_lifo_btc = pd.DataFrame(data_btc, 
                           columns=['Original_Open_index','Original_Close_index',
                                    'Open Position ID', 'Close Position ID',
                                    'Open Units','Close Units','Spread_Eur',
                                    'Open Amount_Eur','Amount_close_Eur',
                                    'Open Datetime', 'Close Datetime',
                                    'Open Rate_Eur', 'Close Rate_Eur',
                                    'Open Action', 'Close Action', 
                                    'Open Amount','Close Rate', 'Open Rate',
                                    'Close Spread', 'Close Profit USD',
                                    'Close_Date','Open_Date',
                                    'Close_unix_timestamp','EurOpen_y', 
                                    'EurHigh_y','EurLow_y', 'EurClose_y',
                                    'EurWeekday_y', 'Open_unix_timestamp',
                                    'EurOpen_x', 'EurHigh_x','EurLow_x', 
                                    'EurClose_x','EurWeekday_x'
                                    ]
                           )

In [2]:
# Let's build a index selector for df_close_btc
close_unit_index = 0
value_close_units = df_close_btc.loc[close_unit_index, 'Close Units']
final_close_unit_index = df_open_btc.shape[0]
fruit_list = list(range(400))
serial_number_for_loop = 0
serial_number_if_loop = 0
serial_number_elif_loop_1 = 0
serial_number_elif_loop_2 = 0
serial_number_else_loop = 0
for x in fruit_list:#while close_unit_index < final_close_unit_index:
  serial_number_for_loop = serial_number_for_loop + 1
  print('For loop run number:',serial_number_for_loop)
  i = 0
  #if value_close_units == 0:
  #  i = 1
  #else:
  #  i = 0
  print('i =',i)
  close_unit_index = close_unit_index + i
  value_close_units = df_close_btc.loc[close_unit_index, 'Close Units']
  # How to set up an upper limit on this variable?? use df.shape[0]
  # Choosing first row from df_close_btc where the units are = values_of_units
  # Assign it as new DF
  # change the reset the index and the drop the 'index' column
  df_latest_row_from_close = df_close_btc[close_unit_index:(close_unit_index+1)]
  df_latest_row_from_close.reset_index(inplace=True)
  current_close_index = df_latest_row_from_close.at[0,'index']
  df_latest_row_from_close = df_latest_row_from_close.drop('index',axis=1)
  # Get the datetime value from the df_latest_row_from_close
  close_unit_datetime = df_close_btc.loc[close_unit_index,'Close Datetime']
  print('close_unit_datetime:',close_unit_datetime)
  # find all the transaction datetimes from df_open that were opened before the close_unit_datetime
  open_datetimes_b4 = df_open_btc['Open Datetime'][df_open_btc['Open Datetime'] < close_unit_datetime]
  #Latest datetime beofore close datetime
  open_datetime_lifo = open_datetimes_b4.max()
  #if open_datetime_lifo == pd.NaT:
  #  break
  #else:
  #  continue
  print('open_datetime_lifo:',open_datetime_lifo)
  '''
  Remember that even if only one value for open_datetime_lifo is chosen
  this can still result in chosing more than one rows in that case last row 
  should be chosen
  '''
  # Corresponding row for the open_datetime_lifo 
  # let's make a new DF
  df_latest_last_row_open = df_open_btc[df_open_btc['Open Datetime'] == open_datetime_lifo]
  df_latest_last_row_open = df_latest_last_row_open[::-1]
  df_latest_last_row_open = df_latest_last_row_open[0:1]
  df_latest_last_row_open.reset_index(inplace=True)
  current_open_index = df_latest_last_row_open.at[0,'index']
  df_latest_last_row_open = df_latest_last_row_open.drop('index', axis=1)
  value_open_units = df_latest_last_row_open.loc[0,'Open Units']
  print('value_close_units:',value_close_units, 'value_open_units:',value_open_units)
  abs_diff_open_close_units = abs(value_close_units - value_open_units)
  print('abs_diff_open_close_units:',abs_diff_open_close_units)
  value_open_amount = df_latest_last_row_open.loc[0,'Open Amount_Eur']
  value_close_amount = df_latest_row_from_close.loc[0,'Amount_close_Eur']
  value_close_spread_eur = df_latest_row_from_close.loc[0,'Spread_Eur']
  close_Rate_eur = df_latest_row_from_close.loc[0,'Close Rate_Eur']
  open_Rate_eur = df_latest_last_row_open.loc[0,'Open Rate_Eur']
  #Remember that 'Open_index' and 'Close_index' columns host original indices
  # This is important as df_open_btc starts to drop rows the original indices will come in handy 
  # And in case of df_close_btc this should remain equal to the indices from DataFrame
  original_open_index = df_latest_last_row_open.at[0, 'Original_Open_index']
  #Remeber that earlier defined close_unit_index is the index of units from df_close_btc DataFrame
  # close_index should essentially be equal in integer value of close_unit_index
  close_index = df_latest_row_from_close.at[0, 'Original_Close_index']
  is_it_equal = close_unit_index == close_index
  print('(close_unit_index = close_index) is:',is_it_equal)
  if value_close_units > value_open_units:
    serial_number_if_loop = serial_number_if_loop + 1
    print('If_loop run number:',serial_number_if_loop)
    #First lets extract all the values from open DF df_latest_last_row_open
    newth_list_open_half = df_latest_last_row_open.to_dict('records')
    newth_dict_open_half = newth_list_open_half[0]
    #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
    df_open_btc = df_open_btc.drop(current_open_index,axis=0)
    #Then reset index and drop the new column 'index' in df_open_btc
    df_open_btc.reset_index(inplace=True)
    df_open_btc = df_open_btc.drop('index',axis=1)
    #Now lets work on the Close_side
    #First define the changed values for df_latest_row_from_close
    changed_value_close_units = value_open_units
    changed_value_close_amount = changed_value_close_units * close_Rate_eur
    changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
    #First change the values of 'units' and 'amount' and 'spread' in the df_latest_row_from_close
    # These values will be reported
    df_latest_row_from_close.at[0,'Close Units'] = changed_value_close_units
    df_latest_row_from_close.at[0,'Amount_close_Eur'] = changed_value_close_amount
    df_latest_row_from_close.at[0,'Spread_Eur'] = changed_value_close_spread
    # Now lets extract all the values from df_latest_row_from_close
    newth_list_close_half = df_latest_row_from_close.to_dict('records')
    newth_dict_close_half = newth_list_close_half[0]
    newth_whole_dict = {**newth_dict_open_half,**newth_dict_close_half}
    # Now append the newth_whole_dict to the new records DataFrame df_lifo_btc
    df_lifo_btc = df_lifo_btc.append(newth_whole_dict, ignore_index=True)
    # and after that change the value of units and amount in original df_open_btc
    df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
    df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
    df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('If_loop run number:',serial_number_if_loop,'finished')
  elif value_close_units < value_open_units:
    serial_number_elif_loop_1 = serial_number_elif_loop_1 + 1
    print('Elif_loop_1 run number:', serial_number_elif_loop_1)
    ## First lets extract all the values form close DF df_latest_row_from_close
    newth_list_close_half = df_latest_row_from_close.to_dict('records')
    newth_dict_close_half = newth_list_close_half[0]
    #First define the changed values for df_latest_row_from_close
    ## In elif statement changed values are same as we made only one change
    changed_value_close_units = value_close_units # value_open_units in if statement was here change to value_close_unit
    changed_value_close_amount = changed_value_close_units * close_Rate_eur
    changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
    # and after that change the value of units and amount and spread in original df_close_btc
    df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
    df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
    df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
    #First change the values of 'units' and 'amount' and 'spread' in the df_latest_row_from_close
    # These values will be reported
    df_latest_row_from_close.at[0,'Close Units'] = changed_value_close_units
    df_latest_row_from_close.at[0,'Amount_close_Eur'] = changed_value_close_amount
    df_latest_row_from_close.at[0,'Spread_Eur'] = changed_value_close_spread
    # Now lets extract all the values from df_latest_row_from_close
    ## Now lets work on the values from open DF df_latest_last_row_lifo
    ## Remember we want to get back the values in the df_latest_row_from_close 
    ## we will have to assign changed_values for open_units and open_amount
    changed_value_open_units = value_close_units
    changed_value_open_amount = changed_value_open_units * open_Rate_eur
    df_latest_last_row_open.at[0,'Open Units'] = changed_value_close_units
    df_latest_last_row_open.at[0,'Open Amount_Eur'] = changed_value_close_amount
    ##After the values are changed extract the values which will then be reported
    newth_list_open_half = df_latest_last_row_open.to_dict('records')
    newth_dict_open_half = newth_list_open_half[0]
    ## and then this time change the original values in the df_open_units
    newth_whole_dict = {**newth_dict_open_half,**newth_dict_close_half}
    # Now append the newth_whole_dict to the new records DataFrame df_lifo_btc
    df_lifo_btc = df_lifo_btc.append(newth_whole_dict, ignore_index=True)
    # and after that change the value of units and amount in original df_open_btc
    df_open_btc.at[current_open_index,'Open Units'] = value_open_units - changed_value_open_units
    df_open_btc.at[current_open_index,'Open Amount_Eur'] = value_open_amount - changed_value_open_amount
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('Elif_loop_1 run number:', serial_number_elif_loop_1, 'finished')
  elif value_close_units == value_open_units:
    serial_number_elif_loop_2 = serial_number_elif_loop_2 + 1
    if value_close_units == 0:
      print('We are running value_close_units == 0 condition')
      print('This is in the Elif_loop_number_2:',serial_number_elif_loop_2)
      #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
      df_open_btc = df_open_btc.drop(current_open_index,axis=0)
      #Then reset index and drop the new column 'index' in df_open_btc
      df_open_btc.reset_index(inplace=True)
      df_open_btc = df_open_btc.drop('index',axis=1)
      print('We are breaking')
      break
    else:
      #First lets extract all the values from open DF df_latest_last_row_open
      newth_list_open_half = df_latest_last_row_open.to_dict('records')
      newth_dict_open_half = newth_list_open_half[0]
      #Then permanently delete the first row of the open DF df_latest_last_row_open from df_open_btc
      df_open_btc = df_open_btc.drop(current_open_index,axis=0)
      #Then reset index and drop the new column 'index' in df_open_btc
      df_open_btc.reset_index(inplace=True)
      df_open_btc = df_open_btc.drop('index',axis=1)
      ### Second lets extract all the values form close DF df_latest_row_from_close
      newth_list_close_half = df_latest_row_from_close.to_dict('records')
      newth_dict_close_half = newth_list_close_half[0]
      #First define the changed values for df_latest_row_from_close
      changed_value_close_units = value_open_units
      changed_value_close_amount = changed_value_close_units * close_Rate_eur
      changed_value_close_spread = changed_value_close_units * value_close_spread_eur/value_close_units
      # and after that change the value of units and amount in original df_open_btc
      df_close_btc.at[close_unit_index,'Close Units'] = value_close_units - changed_value_close_units
      df_close_btc.at[close_unit_index,'Amount_close_Eur'] = value_close_amount - changed_value_close_amount
      df_close_btc.at[close_unit_index,'Spread_Eur'] = value_close_spread_eur - changed_value_close_spread
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('Elif_loop_2 run number:', serial_number_elif_loop_2, 'finished')
  else:
    serial_number_else_loop = serial_number_else_loop + 1
    print('Do we need to write more code? The answer is a: BIG YES!!')
    print('Or on second thought this is a silly mistake!!!')
    print('value_close_units:',value_close_units)
    print('value_open_units:',value_open_units)
    print('close_unit_index:',close_unit_index)
    print('serial_number_else_loop:', serial_number_else_loop, 'finished')
    print('We are breaking here!!')
    break
  value_close_units = df_close_btc.loc[close_unit_index, 'Close Units']
  if value_close_units == 0:
    i = 1
  else:
    i = 0
  print('i =',i)
  close_unit_index = close_unit_index + i
  print('For loop run number:',serial_number_for_loop,'finished')
  print('****************************************')


Streaming output truncated to the last 5000 lines.
abs_diff_open_close_units: 0.069342
(close_unit_index = close_index) is: True
If_loop run number: 40
value_close_units: 0.079878
value_open_units: 0.010536
close_unit_index: 23
If_loop run number: 40 finished
i = 0
For loop run number: 63 finished
****************************************
For loop run number: 64
i = 0
close_unit_datetime: 2020-05-23 20:34:27
open_datetime_lifo: 2020-05-20 16:05:00
value_close_units: 0.069342 value_open_units: 0.052669
abs_diff_open_close_units: 0.016673
(close_unit_index = close_index) is: True
If_loop run number: 41
value_close_units: 0.069342
value_open_units: 0.052669
close_unit_index: 23
If_loop run number: 41 finished
i = 0
For loop run number: 64 finished
****************************************
For loop run number: 65
i = 0
close_unit_datetime: 2020-05-23 20:34:27
open_datetime_lifo: 2020-05-19 19:11:08
value_close_units: 0.016673 value_open_units: 0.162642
abs_diff_open_close_units: 0.1459690000

KeyError: ignored

In [ ]:
df_lifo_btc

In [4]:
df_open_btc

,Original_Open_index,Open Position ID,Open Action,Open Amount,Open Units,Open Datetime,Open Rate,Open_Date,Open_unix_timestamp,EurOpen_x,EurHigh_x,EurLow_x,EurClose_x,EurWeekday_x,Open Amount_Eur,Open Rate_Eur


In [5]:
df_close_btc

,Original_Close_index,Close Position ID,Close Action,Close Units,Close Datetime,Close Rate,Close Spread,Close Profit USD,Close_Date,Close_unix_timestamp,EurOpen_y,EurHigh_y,EurLow_y,EurClose_y,EurWeekday_y,Close Rate_Eur,Amount_close_Eur,Spread_Eur
0,0,599996081,Buy Bitcoin,0.000000e+00,2020-05-01 11:44:24,9027.00,0.96,27.72,01/05/2020 00:00,1588333464,1.094475,1.101625,1.093542,1.094547,Friday,8194.258482,3.818201e-08,0.000000e+00
1,1,610187469,Buy Bitcoin,0.000000e+00,2020-05-01 20:55:45,8685.48,3.83,-30.73,01/05/2020 00:00,1588366545,1.094475,1.101625,1.093542,1.094547,Friday,7884.243731,8.043045e-09,0.000000e+00
2,2,610190271,Buy Bitcoin,0.000000e+00,2020-05-01 20:56:05,8692.19,3.85,-28.20,01/05/2020 00:00,1588366565,1.094475,1.101625,1.093542,1.094547,Friday,7890.334733,-1.377307e-08,-1.387779e-17
3,3,609940706,Buy Bitcoin,0.000000e+00,2020-05-01 20:57:04,8699.43,7.79,-25.13,01/05/2020 00:00,1588366624,1.094475,1.101625,1.093542,1.094547,Friday,7896.906842,-1.747802e-08,0.000000e+00
4,4,609941239,Buy Bitcoin,0.000000e+00,2020-05-01 20:57:54,8709.82,7.37,-31.33,01/05/2020 00:00,1588366674,1.094475,1.101625,1.093542,1.094547,Friday,7906.338364,-3.249613e-08,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,199,1368128599,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:23,27867.01,7.21,-425.77,12/05/2022 00:00,1652346983,1.051248,1.052975,1.037355,1.051248,Thursday,26465.025290,-1.315993e-08,0.000000e+00
200,200,1370705010,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:27,27862.74,3.61,-218.86,12/05/2022 00:00,1652346987,1.051248,1.052975,1.037355,1.051248,Thursday,26460.970110,5.229992e-09,0.000000e+00
201,201,1353186945,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:29,27816.47,0.91,-55.21,12/05/2022 00:00,1652346989,1.051248,1.052975,1.037355,1.051248,Thursday,26417.027940,8.659995e-09,1.110223e-16
202,202,1361963252,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:32,27799.73,0.84,-52.98,12/05/2022 00:00,1652346992,1.051248,1.052975,1.037355,1.051248,Thursday,26401.130130,6.299999e-09,0.000000e+00


In [ ]:
df_lifo_btc.dtypes

Original_Open_index              object
Original_Close_index             object
Open Position ID                 object
Close Position ID                object
Open Units                      float64
Close Units                     float64
Spread_Eur                      float64
Open Amount_Eur                 float64
Amount_close_Eur                float64
Open Datetime            datetime64[ns]
Close Datetime           datetime64[ns]
Open Rate_Eur                   float64
Close Rate_Eur                  float64
Open Action                      object
Close Action                     object
Open Amount                     float64
Close Rate                      float64
Open Rate                       float64
Close Spread                    float64
Close Profit USD                float64
Close_Date                       object
Open_Date                        object
Close_unix_timestamp             object
EurOpen_y                       float64
EurHigh_y                       float64


In [7]:
df_lifo_btc['time_difference'] = df_lifo_btc['Close Datetime'] - df_lifo_btc['Open Datetime']
df_lifo_btc['time_difference_days'] = df_lifo_btc['time_difference'] / np.timedelta64(1, 'D')
df_lifo_btc_taxable = df_lifo_btc[df_lifo_btc['time_difference_days'] < 365]
df_lifo_btc_taxable_2020 = df_lifo_btc_taxable[df_lifo_btc_taxable['Close_unix_timestamp'] <= 1609455599]
df_lifo_btc_taxable_2020_2021 = df_lifo_btc_taxable[df_lifo_btc_taxable['Close_unix_timestamp'] <= 1640991599]
df_lifo_btc_taxable_2021 = df_lifo_btc_taxable_2020_2021[df_lifo_btc_taxable_2020_2021['Close_unix_timestamp'] > 1609455599]
df_lifo_btc_taxable_2022 = df_lifo_btc_taxable[df_lifo_btc_taxable['Close_unix_timestamp'] > 1640991599]
print('Sum of all open positions for the year 2020 in eur:',df_lifo_btc_taxable_2020['Open Amount_Eur'].sum())
print('Sum of all closed positions for the year 2020 in eur:',df_lifo_btc_taxable_2020['Amount_close_Eur'].sum())
print('Sum of all open positions for the year 2021 in eur:',df_lifo_btc_taxable_2021['Open Amount_Eur'].sum())
print('Sum of all closed positions for the year 2021 in eur:',df_lifo_btc_taxable_2021['Amount_close_Eur'].sum())
print('Sum of all open positions for the year 2022 in eur:',df_lifo_btc_taxable_2022['Open Amount_Eur'].sum())
print('Sum of all closed positions for the year 2022 in eur:',df_lifo_btc_taxable_2022['Amount_close_Eur'].sum())

Sum of all open positions for the year 2020 in eur: 23286.8542608751
Sum of all closed positions for the year 2020 in eur: 23431.179723075897
Sum of all open positions for the year 2021 in eur: 21424.486780323197
Sum of all closed positions for the year 2021 in eur: 22841.87929702
Sum of all open positions for the year 2022 in eur: 13984.13312385183
Sum of all closed positions for the year 2022 in eur: 12367.821192006999


In [6]:
df_lifo_btc['time_difference'] = df_lifo_btc['Close Datetime'] - df_lifo_btc['Open Datetime']

In [ ]:
df_lifo_btc['time_difference_days'] = df_lifo_btc['time_difference'] / np.timedelta64(1, 'D')

In [ ]:
df_lifo_btc['time_difference_days']

0        0.687269
1        1.069248
2        1.312211
3        1.695093
4        1.695741
          ...    
380    537.012894
381    537.012940
382    537.509225
383    708.617292
384    754.838380
Name: time_difference_days, Length: 385, dtype: float64

In [ ]:
df_lifo_btc_taxable = df_lifo_btc[df_lifo_btc['time_difference_days'] < 365]

In [ ]:
df_lifo_btc_taxable.shape

(318, 36)

In [ ]:
df_lifo_btc_taxable_2020 = df_lifo_btc_taxable[df_lifo_btc_taxable['Close_unix_timestamp'] <= 1609455599]

In [ ]:
df_lifo_btc_taxable_2020['Open Amount_Eur'].sum()

23286.8542608751

In [ ]:
df_lifo_btc_taxable_2020['Amount_close_Eur'].sum()

23431.179723075897

In [ ]:
df_lifo_btc_2020 = df_lifo_btc[df_lifo_btc['Close_unix_timestamp'] <= 1609455599]

In [ ]:
df_lifo_btc_taxable_2020_2021 = df_lifo_btc_taxable[df_lifo_btc_taxable['Close_unix_timestamp'] <= 1640991599]

In [ ]:
df_lifo_btc_taxable_2021 = df_lifo_btc_taxable_2020_2021[df_lifo_btc_taxable_2020_2021['Close_unix_timestamp'] > 1609455599]

In [ ]:
df_lifo_btc_taxable_2021['Open Amount_Eur'].sum()

21424.486780323197

In [ ]:
df_lifo_btc_taxable_2021['Amount_close_Eur'].sum()

22841.87929702

In [ ]:
df_lifo_btc_taxable_2022 = df_lifo_btc_taxable[df_lifo_btc_taxable['Close_unix_timestamp'] > 1640991599]

In [ ]:
df_lifo_btc_taxable_2022['Open Amount_Eur'].sum()

13984.13312385183

In [ ]:
df_lifo_btc_taxable_2022['Amount_close_Eur'].sum()

12367.821192006999

In [ ]:
df_open_btc

,Original_Open_index,Open Position ID,Open Action,Open Amount,Open Units,Open Datetime,Open Rate,Open_Date,Open_unix_timestamp,EurOpen_x,EurHigh_x,EurLow_x,EurClose_x,EurWeekday_x,Open Amount_Eur,Open Rate_Eur


In [ ]:
df_close_btc

,Original_Close_index,Close Position ID,Close Action,Close Units,Close Datetime,Close Rate,Close Spread,Close Profit USD,Close_Date,Close_unix_timestamp,EurOpen_y,EurHigh_y,EurLow_y,EurClose_y,EurWeekday_y,Close Rate_Eur,Amount_close_Eur,Spread_Eur
0,0,599996081,Buy Bitcoin,0.000000e+00,2020-05-01 11:44:24,9027.00,0.96,27.72,01/05/2020 00:00,1588333464,1.094475,1.101625,1.093542,1.094547,Friday,8194.258482,3.818201e-08,NaN
1,1,610187469,Buy Bitcoin,0.000000e+00,2020-05-01 20:55:45,8685.48,3.83,-30.73,01/05/2020 00:00,1588366545,1.094475,1.101625,1.093542,1.094547,Friday,7884.243731,8.043045e-09,NaN
2,2,610190271,Buy Bitcoin,0.000000e+00,2020-05-01 20:56:05,8692.19,3.85,-28.20,01/05/2020 00:00,1588366565,1.094475,1.101625,1.093542,1.094547,Friday,7890.334733,-1.377307e-08,NaN
3,3,609940706,Buy Bitcoin,0.000000e+00,2020-05-01 20:57:04,8699.43,7.79,-25.13,01/05/2020 00:00,1588366624,1.094475,1.101625,1.093542,1.094547,Friday,7896.906842,-1.747802e-08,NaN
4,4,609941239,Buy Bitcoin,0.000000e+00,2020-05-01 20:57:54,8709.82,7.37,-31.33,01/05/2020 00:00,1588366674,1.094475,1.101625,1.093542,1.094547,Friday,7906.338364,-3.249613e-08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,199,1368128599,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:23,27867.01,7.21,-425.77,12/05/2022 00:00,1652346983,1.051248,1.052975,1.037355,1.051248,Thursday,26465.025290,-1.315993e-08,NaN
200,200,1370705010,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:27,27862.74,3.61,-218.86,12/05/2022 00:00,1652346987,1.051248,1.052975,1.037355,1.051248,Thursday,26460.970110,5.229992e-09,NaN
201,201,1353186945,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:29,27816.47,0.91,-55.21,12/05/2022 00:00,1652346989,1.051248,1.052975,1.037355,1.051248,Thursday,26417.027940,8.659995e-09,NaN
202,202,1361963252,Buy Bitcoin,0.000000e+00,2022-05-12 09:16:32,27799.73,0.84,-52.98,12/05/2022 00:00,1652346992,1.051248,1.052975,1.037355,1.051248,Thursday,26401.130130,6.299999e-09,NaN
